<a href="https://colab.research.google.com/github/kalusyfer/kalusyfer/blob/main/primer_entregable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar librerías
import requests
import json
import pandas as pd
import psycopg2

In [2]:
!pip install requests


In [3]:
import requests

# URL de la API de Blockchain.info para obtener información sobre el último bloque
api_url = "https://blockchain.info/latestblock"

# Realizar la solicitud GET
response = requests.get(api_url)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Si la solicitud fue exitosa (código de estado 200), puedes cargar los datos JSON
    data = response.json()
    print(data)
else:
    # Si hay un problema con la solicitud, imprime un mensaje de error
    print(f"Error al realizar la solicitud. Código de estado: {response.status_code}")


{'block_index': 1706310109, 'hash': '0000000000000000000074ac7d9f54eb70706701d562ee54d5fdac83de51c7a8', 'height': 827540, 'time': 1706310109, 'txIndexes': [8935512595560340, 2196753381739348, 1608360968751506, 2672532770534523, 2188187893946227, 5417935524716209, 5111267217307170, 3524527556262954, 6709100796709659, 5188117494985443, 5520547946449047, 8638689543247748, 1937055436899420, 578271177943472, 1497453291161431, 6666598277330847, 8500932758572431, 8462811724472145, 6859459176828753, 1483478489957621, 7945666292269172, 363201321940769, 7992196102959291, 4940769764331664, 1430329287739095, 6206634311354696, 923219310540117, 8832708446623881, 4706694211879383, 439472842596212, 3016538140560454, 7122554970839642, 3707660782855183, 5218981882127569, 35633136232872, 6957421120127742, 2745583889195369, 8353178407090198, 2260580340942560, 3025523968622928, 4698002444974539, 2418615742913568, 319607895889663, 1035762493844863, 1364074067506162, 2972732908586016, 3981879029588843, 59998

In [4]:
# Convertimos la data a un DataFrame
df = pd.json_normalize(data, sep='_')
# df

# df.columns

# df.dtypes

# df.to_csv('data_entregable_1.csv', index=False)

In [10]:
# conexion base da datos a redshift
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="dev"
user="errementari_cdgd_coderhouse"
pwd = 'wYC955NmX8'

try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")

except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [11]:
# Función para cargar los datos
from psycopg2.extras import execute_values

def cargar_en_redshift(conn, table_name, dataframe):
    dtypes= dataframe.dtypes
    cols= list(dtypes.index )
    tipos= list(dtypes.values)
    type_map = {'int64': 'INT','float64': 'FLOAT','object': 'VARCHAR(50)'}
    sql_dtypes = [type_map[str(dtype)] for dtype in tipos]
    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        );
        """

In [21]:
# Carga de datos en RedShift
cargar_en_redshift(conn=conn, table_name='entregable_1', dataframe=df)

conn.close()